In [1]:
import pandas as pd
import os
import re

In [2]:
# get the root directory, this notebook is in the notebooks directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_files_dir = os.path.join(root_dir, 'data', 'raw')

In [8]:
raw_files = os.listdir(raw_files_dir)
abs_raw_files = [os.path.join(raw_files_dir, f) for f in raw_files]

In [ ]:
abs_raw_files

In [10]:
raw_transactions = pd.read_excel(abs_raw_files[0])

In [ ]:
raw_transactions["Omschrijving"][8]

In [ ]:
raw_transactions["Omschrijving"][12]

In [ ]:
raw_transactions["Omschrijving"][25].split("  ")

In [25]:
transactions = raw_transactions.copy()
def clean_list(lst):
    """Remove empty strings and strings with only spaces."""
    return [item for item in lst if item.strip()]

def parse_transaction(details):
    if details.startswith('/'):
        # Handle structured data
        parts = details.split('/')
        cleaned_parts = clean_list(parts)
        return dict(zip(cleaned_parts[::2], cleaned_parts[1::2]))
    else:
        # Handle unstructured data
        parts = details.split('   ')
        cleaned_parts = clean_list(parts)
        if cleaned_parts[0].startswith('BEA'):
            return {
                'TRTP': cleaned_parts[0],
                'NAME': cleaned_parts[1].split(',')[0].strip(),
                'IBAN': cleaned_parts[1].split(',')[1].strip(),
                'location': cleaned_parts[-1]
            }
    return {}

In [26]:
# Apply the parsing function to the 'Details' column
parsed_details = transactions["Omschrijving"].apply(parse_transaction)

# Convert the parsed details into a DataFrame and concatenate with the original DataFrame
parsed_df = pd.DataFrame(parsed_details.tolist())
transactions = pd.concat([transactions, parsed_df], axis=1)
transactions = transactions.drop(columns=['Omschrijving', "BIC", "EREF"])

In [ ]:
transactions

In [ ]:
transactions = transactions.fillna('NOTPROVIDED')

In [ ]:
def update_name(row):
    # Check if 'tikkie' is in the NAME column (case insensitive)
    if 'tikkie' in row['NAME'].lower():
        if 'via' in row['NAME'].lower():
            # Remove 'via Tikkie' (case insensitive)
            row['NAME'] = re.sub(r'via\s+tikkie', '', row['NAME'], flags=re.IGNORECASE).strip()

            remi_row = row['REMI']
            
            pattern = r'\b\d+\b\s+\b\d+\b\s+(.*?)\s+([A-Z]{2}[A-Z0-9]+)'
            match = re.search(pattern, remi_row)
            if match:
                # The substring is captured in the first group
                substring = match.group(1)
                remi_row = substring.strip()
            else:
                remi_row = None

            row['REMI'] = remi_row
        else:
            # Split the REMI column by comma and take the second to last element
            remi_parts = [part.strip() for part in row['REMI'].split(',')]
            if len(remi_parts) > 1:
                row['NAME'] = remi_parts[-2]
                row['REMI'] = remi_parts[1]
    return row

In [ ]:
transactions = transactions.apply(update_name, axis=1)

In [ ]:
transactions

In [ ]:
# schema of pandas tabke
transactions.dtypes